In [1]:
import config
import json
import requests
import vulture_scraper as vulture
api_key = config.API_key
import time 

In [2]:
## returns full details when given a list of movie IDs
def get_movies(api_key,ids):
    data = []
    for i in ids:
        url = "https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US".format(i,api_key)
        time.sleep(.5)
        data.append(requests.get(url).json())
    return data

In [3]:
## returns movie data when given an API key and list of movie titles 
def search_movies(api_key, list_of_movies):
    data = []
    for movie in list_of_movies[0:40]:
        url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
        data.extend(requests.get(url).json()['results'])
        time.sleep(1)
    #time.sleep(10)
    for movie in list_of_movies[40:]:
        url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
        data.extend(requests.get(url).json()['results'])
        time.sleep(1)
    return data


In [4]:
def search_movies_timed(api_key, list_of_movies):
        lengthToSleep = 40/59
        
        data = []
        
        for movie in list_of_movies:
            url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
            data.append(requests.get(url).json())
            time.sleep(lengthToSleep)
            
        results_list_nested = [x.get('results') for x in data]
        results = [val for sublist in results_list_nested if type(sublist) == list for val in sublist] #<-- if clause makes sure that we're not iterating over none
        
        num_results = sum(filter(None,[x.get('total_results') for x in data])) # <--- not super useful but i was proud of the code
        
        return results 
        

In [5]:
def get_movies_timed(api_key,ids):
    
    lengthToSleep = 40/59        
    
    data = []
    for i in ids:
        url = "https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US".format(i,api_key)
        data.append(requests.get(url).json())
        
        time.sleep(lengthToSleep)
        
    return data

In [7]:
# input list of movies names, return full info for movies
def search_and_get(api_key,list_of_movies):
    search = search_movies_timed(api_key,list_of_movies)
    
    ids = [d['id'] for d in search]
    
    return get_movies_timed(api_key,ids)
        

In [8]:
hulu_results = search_and_get(api_key, vulture.best_hulu)


In [18]:
#print([d['belongs_to_collection'] for d in hulu_results])
for d in hulu_results:
    k_typeValue = [{key:type(key)} for key in d.keys()]
k_typeValue
          

[{'adult': str},
 {'backdrop_path': str},
 {'belongs_to_collection': str},
 {'budget': str},
 {'genres': str},
 {'homepage': str},
 {'id': str},
 {'imdb_id': str},
 {'original_language': str},
 {'original_title': str},
 {'overview': str},
 {'popularity': str},
 {'poster_path': str},
 {'production_companies': str},
 {'production_countries': str},
 {'release_date': str},
 {'revenue': str},
 {'runtime': str},
 {'spoken_languages': str},
 {'status': str},
 {'tagline': str},
 {'title': str},
 {'video': str},
 {'vote_average': str},
 {'vote_count': str}]

In [667]:
## search results will be a list of dictionaries 
def parse_test(search_results):
    parsed_results = {}
    for d in search_results:
        
        parsed_results['id'] = d.get('id')
        parsed_results['collection'] = list(map(lambda x: x['name'],d[]))
        
        if d['budget'] not None:
            parsed_results['budget'] = d.get('budget')
            
        parsed_results['genres'] = [x['name'] for x in d['genres']]
        parsed_results['original_language'] = d.get('orignal_language')
        parsed_results['overview'] d.get('overview')
        parsed_results['title'] = d.get('overview')
        parsed_results['popularity'] = d.get('popularity')
        
        if d['production_comany'] not None:
            parsed_results['production_company'] = [x['name'] for x in d['production_company']]
        
        if d['production_country'] not None:
            parsed_results['production_country']
            
        parsed_results['release_date']
        parsed_results['run_time']
        parsed_results['spoken_language']
        parsed_results['tag_line']
        parsed_results['vote_avg']
        parsed_results['vote_count']
        parsed_results['imdb_id']
        parsed_results['review_count']
        parsed_results['rating']
        
    return parsed_results

In [668]:
parse_test(hulu_results)

{'id': 295836}

In [544]:
## search Movie DB for the best movies on Hulu according to Vulture 

hulu_movies = search_movies(api_key, vulture.best_hulu)

KeyboardInterrupt: 

In [270]:
netflix_movies = search_movies(api_key, vulture.best_netflix)

In [271]:
amazon_movies = search_movies(api_key, vulture.best_amazon)

In [278]:
print(len(hulu_movies)+len(netflix_movies)+len(amazon_movies))
all_ids = {x['id'] for x in hulu_movies}|{x['id'] for x in netflix_movies}|{x['id'] for x in amazon_movies}
len(set((all_ids)))


2132


1958

In [265]:
get_movies_test = get_movies(api_key,set([x['id'] for x in hulu_movies][0:41]))

In [290]:
all_movie_data = get_movies(api_key,set(all_ids))


KeyboardInterrupt: 

In [444]:
type(all_movie_data[0])

dict

In [333]:
df = pd.DataFrame(all_movie_data)
df.head()
df_rev_budget = df[['imdb_id','title','revenue','budget','tagline','runtime','genres']]
df_rev_budget = df_rev_budget[df_rev_budget['revenue']>0]
df_rev_budget['gross_profit']=df['revenue']-df['budget']
df_rev_budget.sort_values('gross_profit',ascending=False,inplace=True)
df_rev_budget

,imdb_id,title,revenue,budget,tagline,runtime,genres,gross_profit
296,tt4154796,Avengers: Endgame,2797800564,356000000,Part of the journey is the end.,181.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",2441800564
736,tt1825683,Black Panther,1346739107,200000000,Long live the king.,134.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1146739107
802,tt2527336,Star Wars: The Last Jedi,1332459537,200000000,Darkness rises... and light to meet it,152.0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",1132459537
1211,tt3606756,Incredibles 2,1241891456,200000000,Back to work.,118.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1041891456
443,tt0298148,Shrek 2,919838758,150000000,Once upon another time...,93.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",769838758
...,...,...,...,...,...,...,...,...
61,tt1046997,Miracle at St. Anna,9323833,45000000,,160.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",-35676167
1479,tt0891592,Street Fighter: The Legend of Chun-Li,12764201,50000000,Some fight for power. Some fight for us.,97.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",-37235799
685,tt0131704,The Adventures of Rocky & Bullwinkle,35134820,76000000,This summer it's not the same old bull.,88.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",-40865180
417,tt0427392,The Invasion,15071514,80000000,Do not trust anyone. Do not show emotion. Do n...,99.0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",-64928486


In [393]:
df_rev_budget['genres']=df_rev_budget['genres'].map(lambda x: [y['id'] for y in x])
df_rev_budget.head()


TypeError: 'int' object is not subscriptable

In [395]:
df['imdb_id'].value_counts()

             108
tt4084278      1
tt0307957      1
tt0053654      1
tt0031534      1
            ... 
tt0106643      1
tt3238502      1
tt6495094      1
tt0370410      1
tt0051850      1
Name: imdb_id, Length: 1670, dtype: int64

In [339]:
df.columns

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [425]:
df[df['imdb_id'].isnull()==True]['imdb_id']

39      None
47      None
49      None
50      None
76      None
        ... 
1905    None
1910    None
1931    None
1949    None
1950    None
Name: imdb_id, Length: 181, dtype: object

In [362]:
df['status'].value_counts()

Released           1932
Planned              15
In Production         8
Post Production       2
Rumored               1
Name: status, dtype: int64

In [504]:
## given a list of dictionaries returned from the 'get_movies' function,
## return a list of parsed results to be entered into AWS DB
## ALSO - you could do this one-by-one. Thoughts?
def parse_movieDB(movies):
    bad_columns = ['video','backdrop_path','homepage','poster_path','adult']
    for d in movies:
        if type(d['belongs_to_collection']) == dict:
            d['belongs_to_collection'] = d['belongs_to_collection'].get('name')
        
        if not d['production_companies']:
            d['production_companies'] = ''
        for elem in d['production_companies']:
            if type(elem) == dict:
                d['production_companies'].append(elem.get('name'))
        
        if not d['spoken_languages']:
            d['spoken_languages'] = ''
        for elem in d['spoken_languages']:
            if type(elem) == dict:
                d['spoken_languages'].append(elem.get('name'))
        
        if not d['production_countries']:
            d['production_countries'] = ''
        for elem in d['production_countries']:
            if type(elem) == dict:
                d['production_countries'].append(elem.get('name'))
                #d['production_countries'] = [x['name'] for x in d['production_countries']]
        
        if not d['genres']:
            d['genres'] = ''
        for elem in d['genres']:
            if type(elem) == dict:
                d['genres'].append(elem.get('name'))
        
        for k in bad_columns:
            if k in d.keys():
                del d[k]
        
        if d['imdb_id'] == 'None':
            del d
        
        elif d['status'] != 'Released':
            del d 
        
    return movies

In [513]:
movie_parse_test = get_movies(api_key,[x['id']for x in hulu_movies][0:10])


In [514]:
movie_parse_test

[{'adult': False,
  'backdrop_path': '/lFiEdW3zVjR2Wujxu3yZLpL2N8n.jpg',
  'belongs_to_collection': {'id': 9380,
   'name': 'American Werewolf Collection',
   'poster_path': '/lz9ThubEYvsKGnXojflsTerhFw8.jpg',
   'backdrop_path': '/Aa3snJ9wKKNkqjU4tAt3Kn4AL2U.jpg'},
  'budget': 10000000,
  'genres': [{'id': 35, 'name': 'Comedy'}, {'id': 27, 'name': 'Horror'}],
  'homepage': None,
  'id': 814,
  'imdb_id': 'tt0082010',
  'original_language': 'en',
  'original_title': 'An American Werewolf in London',
  'overview': 'Two American tourists in England are attacked by a werewolf that none of the locals will admit exists.',
  'popularity': 15.497,
  'poster_path': '/a8doVE7sXezmIeAFyXJcI3hPna5.jpg',
  'production_companies': [{'id': 33,
    'logo_path': '/8lvHyhjr8oUKOOy2dKXoALWKdp0.png',
    'name': 'Universal Pictures',
    'origin_country': 'US'},
   {'id': 276,
    'logo_path': None,
    'name': 'Guber/Peters Company',
    'origin_country': ''},
   {'id': 606,
    'logo_path': None,
    '

In [507]:
clean_items = parse_movieDB(movie_parse_test)
clean_items

[{'belongs_to_collection': 'American Werewolf Collection',
  'budget': 10000000,
  'genres': [{'id': 35, 'name': 'Comedy'},
   {'id': 27, 'name': 'Horror'},
   'Comedy',
   'Horror'],
  'id': 814,
  'imdb_id': 'tt0082010',
  'original_language': 'en',
  'original_title': 'An American Werewolf in London',
  'overview': 'Two American tourists in England are attacked by a werewolf that none of the locals will admit exists.',
  'popularity': 15.497,
  'production_companies': [{'id': 33,
    'logo_path': '/8lvHyhjr8oUKOOy2dKXoALWKdp0.png',
    'name': 'Universal Pictures',
    'origin_country': 'US'},
   {'id': 276,
    'logo_path': None,
    'name': 'Guber/Peters Company',
    'origin_country': ''},
   {'id': 606,
    'logo_path': None,
    'name': 'American Werewolf Inc.',
    'origin_country': ''},
   {'id': 607,
    'logo_path': None,
    'name': 'Lyncanthrope Films',
    'origin_country': ''},
   'Universal Pictures',
   'Guber/Peters Company',
   'American Werewolf Inc.',
   'Lyncanth

In [515]:
def parse_test(movies):
    for d in movies:
        d['genres'] = list(map(lambda x: x['name'],d['genres']))
    return movies
test = parse_test(movie_parse_test)
test

[{'adult': False,
  'backdrop_path': '/lFiEdW3zVjR2Wujxu3yZLpL2N8n.jpg',
  'belongs_to_collection': {'id': 9380,
   'name': 'American Werewolf Collection',
   'poster_path': '/lz9ThubEYvsKGnXojflsTerhFw8.jpg',
   'backdrop_path': '/Aa3snJ9wKKNkqjU4tAt3Kn4AL2U.jpg'},
  'budget': 10000000,
  'genres': ['Comedy', 'Horror'],
  'homepage': None,
  'id': 814,
  'imdb_id': 'tt0082010',
  'original_language': 'en',
  'original_title': 'An American Werewolf in London',
  'overview': 'Two American tourists in England are attacked by a werewolf that none of the locals will admit exists.',
  'popularity': 15.497,
  'poster_path': '/a8doVE7sXezmIeAFyXJcI3hPna5.jpg',
  'production_companies': [{'id': 33,
    'logo_path': '/8lvHyhjr8oUKOOy2dKXoALWKdp0.png',
    'name': 'Universal Pictures',
    'origin_country': 'US'},
   {'id': 276,
    'logo_path': None,
    'name': 'Guber/Peters Company',
    'origin_country': ''},
   {'id': 606,
    'logo_path': None,
    'name': 'American Werewolf Inc.',
    'or

In [536]:
def parse_test2(movies):
    unwanted_columns = ['genres',"production_companies",'production_countries']
    for d in movies:
        for column in unwanted_columns:
            d[column]=list(map(lambda x: x['name'],d[column]))
    return movies

In [538]:
test = parse_test2(movie_parse_test)


TypeError: string indices must be integers

In [539]:
movie_parse_test

[{'adult': False,
  'backdrop_path': '/lFiEdW3zVjR2Wujxu3yZLpL2N8n.jpg',
  'belongs_to_collection': {'id': 9380,
   'name': 'American Werewolf Collection',
   'poster_path': '/lz9ThubEYvsKGnXojflsTerhFw8.jpg',
   'backdrop_path': '/Aa3snJ9wKKNkqjU4tAt3Kn4AL2U.jpg'},
  'budget': 10000000,
  'genres': ['Comedy', 'Horror'],
  'homepage': None,
  'id': 814,
  'imdb_id': 'tt0082010',
  'original_language': 'en',
  'original_title': 'An American Werewolf in London',
  'overview': 'Two American tourists in England are attacked by a werewolf that none of the locals will admit exists.',
  'popularity': 15.497,
  'poster_path': '/a8doVE7sXezmIeAFyXJcI3hPna5.jpg',
  'production_companies': [{'id': 33,
    'logo_path': '/8lvHyhjr8oUKOOy2dKXoALWKdp0.png',
    'name': 'Universal Pictures',
    'origin_country': 'US'},
   {'id': 276,
    'logo_path': None,
    'name': 'Guber/Peters Company',
    'origin_country': ''},
   {'id': 606,
    'logo_path': None,
    'name': 'American Werewolf Inc.',
    'or